In [1]:
# 0) 환경 세팅 및 라이브러리 설치

# Hugging Face 라이브러리 설치
!pip install huggingface_hub ultralytics
!pip install doclayout-yolo
!git clone https://github.com/opendatalab/DocLayout-YOLO.git

# 필요한 시스템 패키지 및 라이브러리 설치
!apt-get update
!apt-get install -y poppler-utils  # PDF → 이미지 변환에 필요한 poppler
!pip install torch transformers==4.40.0 accelerate
!pip install pdf2image            # PDF → 이미지 변환 라이브러리
!pip install --upgrade transformers tokenizers huggingface_hub
!pip install easyocr              # OCR을 위한 easyocr

fatal: destination path 'DocLayout-YOLO' already exists and is not an empty directory.
'apt-get'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'apt-get'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


  Using cached transformers-4.40.0-py3-none-any.whl.metadata (137 kB)
  Using cached tokenizers-0.19.1-cp312-none-win_amd64.whl.metadata (6.9 kB)
Using cached transformers-4.40.0-py3-none-any.whl (9.0 MB)
Using cached tokenizers-0.19.1-cp312-none-win_amd64.whl (2.2 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.2
    Uninstalling transformers-4.48.2:
      Successfully uninstalled transformers-4.48.2


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


  Using cached transformers-4.48.2-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
Using cached transformers-4.48.2-py3-none-any.whl (9.7 MB)
Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl (2.4 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0
    Uninstalling transformers-4.40.0:
      Successfully uninstalled transformers-4.40.0


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [1]:
# 1) 필요한 Python 라이브러리 임포트

from huggingface_hub import hf_hub_download
from ultralytics import YOLO
import numpy as np
import pandas as pd
import cv2
import json
from doclayout_yolo import YOLOv10  # DocLayout-YOLO
import os
import uuid
from pdf2image import convert_from_path
import easyocr
from collections import Counter
from tqdm import tqdm
import transformers
import torch

In [2]:
# 2) PDF → 이미지 변환 함수 정의

def pdf_to_jpg(pdf_path, output_folder="images", dpi=300):
    """
    PDF 파일을 지정된 DPI로 페이지별 JPG 이미지로 변환하고,
    변환된 이미지 경로 리스트를 반환한다.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # PDF를 이미지로 변환
    images = convert_from_path(pdf_path, dpi=dpi)

    image_paths = []
    for i, image in enumerate(images):
        image_path = os.path.join(output_folder, f"page_{i+1}.jpg")
        image.save(image_path, "JPEG")
        image_paths.append(image_path)

    print(f"Converted PDF to {len(image_paths)} JPG files.")
    return image_paths

In [3]:
# 3) 바운딩 박스 처리 (IoU, 중복 제거 등)

def calculate_iou(box1, box2):
    """
    두 바운딩 박스 사이의 IoU(Intersection over Union)를 계산한다.
    box1, box2: (x_min, y_min, x_max, y_max) 형태의 좌표 튜플
    """
    x1, y1, x2, y2 = box1
    x3, y3, x4, y4 = box2

    # 교집합 영역 좌표 계산
    inter_x1 = max(x1, x3)
    inter_y1 = max(y1, y3)
    inter_x2 = min(x2, x4)
    inter_y2 = min(y2, y4)

    # 교집합 면적
    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)

    # 각 바운딩 박스 면적
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x4 - x3) * (y4 - y3)

    # IoU 계산
    return inter_area / (box1_area + box2_area - inter_area)

def filter_duplicate_boxes(bounding_boxes, iou_threshold=0.5):
    """
    IoU 기반으로 중복된 바운딩 박스를 제거하는 함수.
    두 박스가 iou_threshold 이상 겹치면 더 높은 신뢰도(confidence)만 남긴다.
    """
    filtered_boxes = []
    for box in bounding_boxes:
        keep = True
        for fbox in filtered_boxes:
            iou = calculate_iou(
                (box["x_min"], box["y_min"], box["x_max"], box["y_max"]),
                (fbox["x_min"], fbox["y_min"], fbox["x_max"], fbox["y_max"])
            )
            if iou > iou_threshold:
                # 더 높은 confidence를 가진 박스만 남김
                if box["confidence"] > fbox["confidence"]:
                    filtered_boxes.remove(fbox)
                else:
                    keep = False
                break
        if keep:
            filtered_boxes.append(box)
    return filtered_boxes

def generate_unique_suffix(index):
    """
    주어진 인덱스를 기반으로 영문 소문자(a-z) 중 하나를 반환한다.
    인덱스를 26으로 나눈 나머지에 따라 a~z를 할당한다.
    """
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    return alphabet[index % len(alphabet)]

In [4]:
# 4) YOLO 모델 추론 후 바운딩 박스 JSON 저장

def process_image(image_path, model, page_number, output_folder="output_results"):
    """
    1) YOLO 모델로 이미지 내 객체(표, 그림 등)를 검출
    2) 중복 박스 제거 후 JSON으로 저장
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 모델 예측 수행 (conf=0.2는 confidence threshold)
    det_res = model.predict(image_path, imgsz=1024, conf=0.2, device="cpu")

    # 추론 결과를 기반으로 바운딩 박스 리스트 생성
    bounding_boxes = []
    for i, box in enumerate(det_res[0].boxes):
        class_name = model.names[int(box.cls)]  # 클래스 이름(예: table, figure)
        class_number = int(box.cls)            # 클래스 번호
        unique_suffix = generate_unique_suffix(i)

        bounding_boxes.append({
            "class": class_name,
            "confidence": float(box.conf),
            "x_min": float(box.xyxy[0][0]),
            "y_min": float(box.xyxy[0][1]),
            "x_max": float(box.xyxy[0][2]),
            "y_max": float(box.xyxy[0][3]),
            "unique_id": f"{page_number}_{class_number}_{unique_suffix}"  # 페이지번호_클래스번호_고유문자
        })

    # IoU 기반 중복 제거
    filtered_boxes = filter_duplicate_boxes(bounding_boxes, iou_threshold=0.5)

    # JSON 파일로 저장
    json_output_path = os.path.join(output_folder, f"page_{page_number}_filtered_boxes.json")
    with open(json_output_path, "w") as f:
        json.dump(filtered_boxes, f, indent=4)
    print(f"Saved filtered boxes for Page {page_number} to: {json_output_path}")

In [5]:
# 5) YOLO 모델 로드 및 메인 실행 함수

def load_yolo_model():
    """
    Hugging Face Hub에서 DocLayout-YOLO 모델 가중치를 다운로드 받아 YOLOv10 모델 객체를 반환한다.
    """
    filepath = hf_hub_download(
        repo_id="juliozhao/DocLayout-YOLO-DocStructBench",
        filename="doclayout_yolo_docstructbench_imgsz1024.pt"
    )
    return YOLOv10(filepath)

def main(pdf_path, output_folder="output_results"):
    """
    1) YOLO 모델 로딩
    2) PDF를 페이지별 이미지를 변환
    3) 각 페이지 이미지를 YOLO 추론하여 결과 JSON 저장
    """
    print("Loading YOLO model...")
    model = load_yolo_model()

    print("Converting PDF to images...")
    image_paths = pdf_to_jpg(pdf_path, output_folder="images")

    print("Processing images...")
    for page_number, image_path in enumerate(image_paths, start=1):
        process_image(image_path, model, page_number, output_folder)

In [6]:
# 6) PDF → 이미지 + YOLO 바운딩 박스 추출 실행

if __name__ == "__main__":
    pdf_path = "비타민 CV 프로젝트.pdf"  # 처리할 PDF 경로 (사용자 지정)
    main(pdf_path)

Loading YOLO model...
Converting PDF to images...
Converted PDF to 2 JPG files.
Processing images...

image 1/1 c:\Workspace\images\page_1.jpg: 1024x736 2 titles, 3 plain texts, 1 abandon, 2 figures, 2 figure_captions, 3102.9ms
Speed: 10.9ms preprocess, 3102.9ms inference, 0.0ms postprocess per image at shape (1, 3, 1024, 736)
Saved filtered boxes for Page 1 to: output_results\page_1_filtered_boxes.json

image 1/1 c:\Workspace\images\page_2.jpg: 1024x736 1 plain text, 1 abandon, 2 tables, 3062.1ms
Speed: 7.6ms preprocess, 3062.1ms inference, 2.0ms postprocess per image at shape (1, 3, 1024, 736)
Saved filtered boxes for Page 2 to: output_results\page_2_filtered_boxes.json


In [7]:
# 7) 클래스별 JSON 파일 통합 (table, figure 등)

def combine_json_by_class(directory_path, output_table_file="table_json.json", output_figure_file="figure_json.json"):
    """
    output_results 폴더 내 모든 *_filtered_boxes.json 파일을 확인하여
    class=='table' / class=='figure'로 분류한 뒤 각각 table_json.json, figure_json.json에 저장한다.
    """
    table_data = []
    figure_data = []

    # *_filtered_boxes.json 파일 찾아서 반복
    json_files = [f for f in os.listdir(directory_path) if f.endswith("_filtered_boxes.json")]

    for json_file in json_files:
        json_file_path = os.path.join(directory_path, json_file)
        with open(json_file_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        # 클래스에 따라 table_data, figure_data에 나누어 담기
        for box in data:
            if box["class"] == "table":
                table_data.append(box)
            elif box["class"] == "figure":
                figure_data.append(box)

    # 테이블 JSON, 그림 JSON 각각 저장
    save_json(os.path.join(directory_path, output_table_file), table_data, "table")
    save_json(os.path.join(directory_path, output_figure_file), figure_data, "figure")

def save_json(output_path, data, data_type):
    """
    data를 JSON 파일로 저장. 테이블/그림 등을 구분하기 위해 data_type 사용.
    """
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)
    print(f"{data_type.capitalize()} data saved to: {output_path}")

# 결합 실행 예시
directory_path = "output_results"  # YOLO 바운딩 박스 결과 JSON 폴더
combine_json_by_class(directory_path)

Table data saved to: output_results\table_json.json
Figure data saved to: output_results\figure_json.json


In [8]:
# 8) JSON 바운딩 박스를 바탕으로 이미지 크롭 & 저장

def crop_and_save_by_json(image_dir, json_path, output_dir):
    """
    json_path에 정의된 바운딩 박스 좌표를 사용해
    image_dir의 원본 이미지를 잘라내어(output_dir에) 저장한다.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 바운딩 박스 정보 로드
    with open(json_path, "r", encoding="utf-8") as f:
        bounding_boxes = json.load(f)

    # 각 박스별로 이미지를 잘라(crop)서 저장
    for box in bounding_boxes:
        # unique_id에서 페이지 번호 추출
        page_number = int(box["unique_id"].split("_")[0])
        image_name = f"page_{page_number}.jpg"
        image_path = os.path.join(image_dir, image_name)

        image = cv2.imread(image_path)
        if image is None:
            print(f"이미지를 불러올 수 없습니다: {image_path}")
            continue

        x_min = int(box["x_min"])
        y_min = int(box["y_min"])
        x_max = int(box["x_max"])
        y_max = int(box["y_max"])

        cropped_image = image[y_min:y_max, x_min:x_max]

        # crop된 이미지를 고유 ID를 파일명 삼아 저장
        save_path = os.path.join(output_dir, f"{box['unique_id']}.jpg")
        cv2.imwrite(save_path, cropped_image)
        print(f"Saved cropped image to: {save_path}")

# 테이블/그림 크롭 실행 예시
image_dir = "images"               # PDF → JPG 변환된 페이지 이미지 폴더
output_dir_base = "cropped_images" # 잘라낸 이미지가 저장될 기본 폴더

# figure / table 각각 크롭
json_files = {
    "figure": "output_results/figure_json.json",
    "table": "output_results/table_json.json"
}

for category, json_path in json_files.items():
    output_dir = os.path.join(output_dir_base, category)
    crop_and_save_by_json(image_dir, json_path, output_dir)

Saved cropped image to: cropped_images\figure\1_3_c.jpg
Saved cropped image to: cropped_images\figure\1_3_e.jpg
Saved cropped image to: cropped_images\table\2_5_a.jpg
Saved cropped image to: cropped_images\table\2_5_c.jpg


In [9]:
import torch
import easyocr
import cv2
import numpy as np
import json
import os
from tqdm import tqdm

class ImageProcessor:
    """
    전처리를 위한 클래스.
    (그레이스케일, 블러, 이진화, 모폴로지 연산 등)
    """
    def __init__(self, image):
        self.image = image
        if self.image is None:
            raise ValueError("Could not load image")

    def preprocess(self):
        """
        GPU 가속을 활용한 이미지 전처리 (그레이스케일, 블러, 이진화)
        """
        if len(self.image.shape) == 3:
            self.gray = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)
        else:
            self.gray = self.image

        # OpenCV CUDA 사용 가능 여부 확인
        self.use_cuda = cv2.cuda.getCudaEnabledDeviceCount() > 0
        print(f"🔹 OpenCV CUDA 사용 여부: {self.use_cuda}")

        if self.use_cuda:
            gpu_img = cv2.cuda_GpuMat()
            gpu_img.upload(self.gray)
            gpu_blurred = cv2.cuda.GaussianBlur(gpu_img, (5, 5), 0)
            self.blurred = gpu_blurred.download()
        else:
            self.blurred = cv2.GaussianBlur(self.gray, (5, 5), 0)

        _, self.binary = cv2.threshold(self.blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        return self.binary

    def apply_morphology(self):
        """
        모폴로지 연산을 통해 표의 선(가로/세로 선 등)을 추출하는 예시 (선택적)
        """
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50, 1))
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 50))

        self.horizontal_lines = cv2.morphologyEx(self.binary, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
        self.vertical_lines = cv2.morphologyEx(self.binary, cv2.MORPH_OPEN, vertical_kernel, iterations=2)
        self.morph_result = cv2.bitwise_or(self.horizontal_lines, self.vertical_lines)
        return self.morph_result

class TextExtractor:
    """
    EasyOCR 기반으로 이미지를 읽어 텍스트를 추출하는 클래스.
    """
    def __init__(self, language=['ko']):
        self.use_gpu = torch.cuda.is_available()
        print(f"EasyOCR GPU 사용 여부: {self.use_gpu}")

        self.reader = easyocr.Reader(language, gpu=self.use_gpu)

    def extract_text(self, image):
        """이미지로부터 텍스트를 추출하여 리턴"""
        text_result = self.reader.readtext(image, detail=0)
        return "\n".join(text_result).strip()

def process_images_in_directory(input_dir, output_dir):
    """
    input_dir 내의 모든 이미지에 대해 전처리 + OCR 수행 후,
    추출된 텍스트를 JSON으로 저장한다.
    """
    if not os.path.exists(input_dir):
        raise ValueError(f"Input directory does not exist: {input_dir}")

    results = []

    # 폴더 내 모든 이미지 파일 반복 처리
    for file_name in tqdm(os.listdir(input_dir), desc="Processing Images"):
        if file_name.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff")):
            image_path = os.path.join(input_dir, file_name)

            image = cv2.imread(image_path)
            if image is None:
                print(f"Failed to read image: {image_path}")
                continue

            # 이미지 전처리 & OCR
            try:
                processor = ImageProcessor(image)
                binary = processor.preprocess()
                processor.apply_morphology()

                extractor = TextExtractor()
                extracted_text = extractor.extract_text(image)

                results.append({
                    "file_name": file_name,
                    "extracted_text": extracted_text
                })

                # GPU 메모리 캐시 정리
                torch.cuda.empty_cache()

            except Exception as e:
                print(f"Error processing {file_name}: {e}")

    # 최종 OCR 결과 JSON으로 저장
    save_results_as_json(results, output_dir)

def save_results_as_json(results, output_dir):
    """
    OCR 추출 결과를 JSON 파일로 저장.
    """
    os.makedirs(output_dir, exist_ok=True)
    results_json_path = os.path.join(output_dir, "result_table.json")
    with open(results_json_path, "w", encoding="utf-8") as json_file:
        json.dump(results, json_file, ensure_ascii=False, indent=4)
    print(f" Extracted text saved to: {results_json_path}")

# 표 이미지를 OCR 실행 예시
input_dir = "cropped_images/table"
output_dir = "outputs"
process_images_in_directory(input_dir, output_dir)

Processing Images:   0%|          | 0/2 [00:00<?, ?it/s]Using CPU. Note: This module is much faster with a GPU.


🔹 OpenCV CUDA 사용 여부: False
🔥 EasyOCR GPU 사용 여부: False


Processing Images:  50%|█████     | 1/2 [00:23<00:23, 23.15s/it]Using CPU. Note: This module is much faster with a GPU.


🔹 OpenCV CUDA 사용 여부: False
🔥 EasyOCR GPU 사용 여부: False


Processing Images: 100%|██████████| 2/2 [00:34<00:00, 17.35s/it]

✅ Extracted text saved to: outputs\result_table.json


: 

In [12]:
# 10) 그림(차트) 이미지(Pix2Struct) 처리
#  - brainventures/ko-deplot 모델 사용

from transformers import Pix2StructProcessor, Pix2StructForConditionalGeneration
import torch
from datetime import datetime
from pytz import timezone
import os
import json
from tqdm import tqdm
import warnings
from PIL import Image

warnings.filterwarnings('ignore')

MAX_PATCHES = 512  # Pix2Struct에서 사용할 max_patches 파라미터

class ImageToDataTable:
    """
    Hugging Face Hub에 있는 'brainventures/deplot_kr' 모델을 사용하여,
    그래프/차트 이미지를 구조화된 텍스트(데이터 테이블 등)로 변환하는 클래스.
    """
    def __init__(self, model_id: str = "brainventures/deplot_kr", output_dir: str = "outputs", device: str = None):
        # 디바이스 설정 (GPU가 있으면 cuda, 없으면 cpu)
        self.device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")
        self.output_dir = output_dir
        self.model_id = model_id

        # GPU 사용 여부 확인
        print(f"Using device: {self.device.upper()}")

        # 모델 로드: 한국어 DePlot (brainventures/deplot_kr)
        self.processor = Pix2StructProcessor.from_pretrained(model_id)
        self.model = Pix2StructForConditionalGeneration.from_pretrained(model_id)

        # 모델을 지정한 디바이스로 이동
        self.model.to(self.device)
        self.model.eval()

        # 결과 저장 디렉토리
        os.makedirs(self.output_dir, exist_ok=True)

    def process_image(self, image_path: str, prompt: str = None):
        """
        단일 이미지를 Pix2Struct로 추론하여 결과 텍스트를 반환.
        prompt: 모델에게 '데이터 테이블을 생성하라'는 지시문 등. (기본 영문 프롬프트 예시)
        """
        if prompt is None:
            prompt = "Generate underlying data table of the figure below:"

        data_id = os.path.splitext(os.path.basename(image_path))[0]

        # 이미지 로드
        try:
            image = Image.open(image_path).convert("RGB")
        except Exception as e:
            print(f"[Error] Failed to open image {data_id}: {e}")
            return None, None

        # 모델 입력 구성
        inputs = self.processor(images=image, text=prompt, return_tensors="pt", max_patches=MAX_PATCHES)
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        # 모델 추론
        with torch.no_grad():
            generated_ids = self.model.generate(**inputs, max_new_tokens=512)

        # 결과 텍스트 디코딩
        generated_text = self.processor.decode(generated_ids[0], skip_special_tokens=True)

        # prompt 문장을 제거한 결과 반환
        clean_text = generated_text[len(prompt):].strip()
        return data_id, clean_text

    def process_images(self, image_paths, prompt: str = None):
        """
        여러 이미지를 한 번에 처리하고, JSON으로 결과를 저장한다.
        """
        log_path = os.path.join(self.output_dir, "evaluation_log.txt")
        json_data = []

        # 시작 시간 로그
        with open(log_path, "a", encoding='utf-8') as log_file:
            current_time = datetime.now(timezone('Asia/Seoul'))
            print(f"[Start] {current_time} / Total images: {len(image_paths)}")
            log_file.write(f"Start: {current_time} / Data: {len(image_paths)}\n")

            for image_path in tqdm(image_paths, desc="Processing Images"):
                data_id, clean_text = self.process_image(image_path, prompt)
                if clean_text:
                    json_data.append({
                        "data_id": data_id,
                        "generated_text": clean_text  # prompt 제거된 순수 생성문 저장
                    })

            current_time = datetime.now(timezone('Asia/Seoul'))
            print(f"[End] {current_time} / Processed: {len(json_data)}")
            log_file.write(f"End: {current_time} / Data: {len(json_data)}\n")

        # 결과 저장
        self._save_results(json_data)

    def _save_results(self, json_data):
        """
        결과를 result_figure.json으로 저장.
        """
        json_output_path = os.path.join(self.output_dir, "result_figure.json")
        with open(json_output_path, "w", encoding='utf-8') as jf:
            json.dump(json_data, jf, ensure_ascii=False, indent=4)
        print(f"[Done] JSON results saved to: {json_output_path}")


def get_image_paths(data_path: str):
    """
    지정된 폴더에서 지원되는 확장자(.jpg, .png 등) 파일들의 경로를 수집해 리스트로 반환
    """
    supported_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.gif')
    return [
        os.path.join(data_path, fname)
        for fname in os.listdir(data_path)
        if fname.lower().endswith(supported_extensions)
    ]

# ------------ 실행 예시 -------------
DEFAULT_DATA_PATH = "/content/cropped_images/figure"  # 그림(차트) 이미지가 들어있는 폴더
DEFAULT_OUTPUT_DIR = "/content/outputs"               # 결과(JSON) 저장 폴더

if __name__ == "__main__":
    # 1) figure 폴더 내의 이미지 경로 수집
    image_paths = get_image_paths(DEFAULT_DATA_PATH)
    if not image_paths:
        raise ValueError(f"No supported image files found in {DEFAULT_DATA_PATH}")

    # 2) brainventures/deplot_kr 모델로 초기화
    image_to_table = ImageToDataTable(
        model_id="brainventures/deplot_kr",
        output_dir=DEFAULT_OUTPUT_DIR
    )

    # 3) 일괄 처리 (프롬프트는 원하는 문구로 교체 가능)
    image_to_table.process_images(
        image_paths=image_paths,
        prompt="다음 차트의 데이터 표를 한국어로 상세히 출력해줘:"
    )

Using device: CPU
[Start] 2025-02-02 13:12:34.412815+09:00 / Total images: 2


Processing Images: 100%|██████████| 2/2 [03:03<00:00, 91.72s/it]

[End] 2025-02-02 13:15:37.868168+09:00 / Processed: 2
[Done] JSON results saved to: /content/outputs/result_figure.json
